In [1]:
import pandas as pd
import numpy as np

In [2]:
# Import Raw Data from excel files into dataFrames
df10=pd.read_excel('mlb_odds_2010.xlsx')
df10['Year'] = 2010
df11=pd.read_excel('mlb_odds_2011.xlsx')
df11['Year'] = 2011
df12=pd.read_excel('mlb_odds_2012.xlsx')
df12['Year'] = 2012
df13=pd.read_excel('mlb_odds_2013.xlsx')
df13['Year'] = 2013
df14=pd.read_excel('mlb_odds_2014.xlsx')
df14['Year'] = 2014
df15=pd.read_excel('mlb_odds_2015.xlsx')
df15['Year'] = 2015
df16=pd.read_excel('mlb_odds_2016.xlsx')
df16['Year'] = 2016
df17=pd.read_excel('mlb_odds_2017.xlsx')
df17['Year'] = 2017
df18=pd.read_excel('mlb_odds_2018.xlsx')
df18['Year'] = 2018
df19=pd.read_excel('mlb_odds_2019.xlsx')
df19['Year'] = 2019

In [3]:
# make list of dataFrames
dfList = [df10, df11, df12, df13, df14, df15, df16, df17, df18, df19]

In [7]:
# reformat data
for df in dfList:
    try:
        dfnew=df['RL'].str.split(pat="(", n=1, expand=True)
        dfnewer = dfnew[1].str.rstrip("\)")
        df['RunLine'] = dfnew[0].apply(float)
        df['RunLineOdds'] = dfnewer.apply(int)
        df.rename({"Unnamed: 18":"OpenOUOdds", "Unnamed: 19":"OpenOUOdds", "closeou":"close_ou", "Unnamed: 20":"CloseOUOdds", "Unnamed: 21":"CloseOUOdds"}, axis='columns', inplace=True)
    except:
        df.rename({"Unnamed: 18":"OpenOUOdds", "Unnamed: 19":"OpenOUOdds", "closeou":"close_ou", "Unnamed: 20":"CloseOUOdds", "Unnamed: 21":"CloseOUOdds"}, axis='columns', inplace=True)
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
for df in dfList:
    colNew = []
    i = 0
    for index, row in df.iterrows():
        if index == 0:
            pass
        elif index % 2 == 0:
            i += 1
        colNew.append(i)
    df['gameno'] = colNew

In [10]:
def convt(num):
    if type(num) == str:
        num = float(num[0]) + .5
    return num
for df in dfList:
    df['close_ou'] = df['close_ou'].apply(convt)

In [12]:
# create additional columns
for df in dfList:
    totRuns = []
    counter = 0
    try:
        for index, row in df.iterrows():
            if row.final == "NL":
                totRuns.append(row.final)
            elif index % 2 == 0:
                counter = 0
                counter += row.final
            else:
                counter += row.final
                totRuns.append(counter)
                totRuns.append(counter)
    except TypeError:
        print(row)
    df['totalruns'] = totRuns
    RunDif = []
    counter = 0
    try:
        for index, row in df.iterrows():
            if row.final == "NL":
                RunDif.append(row.final)
            elif index % 2 == 0:
                counter = 0
                team1 = row.final
            else:
                team2 = row.final
                dif = team1 - team2
                RunDif.append(dif)
                RunDif.append(-dif)
    except TypeError:
        print(row)
    df['rundif'] = RunDif
    winRL = []
    for index, row in df.iterrows():
        # favorite, win by more than 1.5 = True
        # favorite, win by less than 1.5 or lose = False
        # underdog, wins or loses by less than 1.5 = True
        # underdog, loses by more than 1.5 = True
        try:
            if row.runline < 0:
                if row.rundif > abs(row.runline):
                    winRL.append(True)
                else:
                    winRL.append(False)
            else:
                if row.rundif > 0 or abs(row.rundif) < row.runline:
                    winRL.append(True)
                else:
                    winRL.append(False)
        except AttributeError:
            winRL.append("NaN")
        except TypeError:
            winRL.append("NaN")
        except ValueError:
            winRL.append("NaN")
    df['winrl'] = winRL
    winTot = []
    for index, row in df.iterrows():
        try:
            if index % 2 == 0:
                if row.totalruns > row.close_ou:
                    winTot.append(True)
                else:
                    winTot.append(False)
            else:
                if row.totalruns > row.close_ou:
                    winTot.append(False)
                else:
                    winTot.append(True)
        except TypeError:
            winTot.append(True)
    df['wintotal'] = winTot

In [13]:
#TODO
Start linear regression model

,date,rot,vh,team,pitcher,1st,2nd,3rd,4th,5th,...,close_ou,closeouodds,year,runline,runlineodds,gameno,totalruns,rundif,winrl,wintotal
0,329,901,V,CUB,JLESTER-L,3,1,0,1,0,...,8.0,-120,2018,-1.5,-125,0,12,4,True,True
1,329,902,H,MIA,JURENA-R,1,0,3,0,0,...,8.0,100,2018,1.5,105,0,12,-4,False,False
2,329,903,V,STL,CMARTINEZ-R,0,2,0,1,0,...,6.5,-125,2018,1.5,-185,1,13,-5,False,True
3,329,904,H,NYM,NSYNDERGAARD-R,1,2,0,0,5,...,6.5,105,2018,-1.5,160,1,13,5,True,False
4,329,907,V,PHI,ANOLA-R,1,0,0,0,0,...,8.0,-119,2018,-1.5,125,2,13,-3,False,True
5,329,908,H,ATL,JTEHERAN-R,0,0,0,0,0,...,8.0,-101,2018,1.5,-145,2,13,3,True,False
6,329,909,V,MIL,CANDERSON-R,0,0,1,0,0,...,8.0,-115,2018,-1.5,130,3,3,1,False,False
7,329,910,H,SDG,CRICHARD-L,0,0,0,0,0,...,8.0,-105,2018,1.5,-150,3,3,-1,True,True
8,329,911,V,SFO,TBLACH-L,0,0,0,0,1,...,7.5,-116,2018,1.5,125,4,1,1,True,False
9,329,912,H,LAD,CKERSHAW-L,0,0,0,0,0,...,7.5,-104,2018,-1.5,-145,4,1,-1,False,True
